In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
from simulation import minimal_model as mm
from surrogate import neural_network

from sampling import grid, random, lhs
from visualise import stream, surface

# EGO test

In [ ]:
import numpy as np
from scipy.optimize import minimize
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.preprocessing import StandardScaler
from scipy.stats import norm  # Add this import

class EGO:
    def __init__(self, func, bounds, initial_points, noise=0.1):
        self.func = func
        self.bounds = bounds
        self.noise = noise
        self.X = np.array(initial_points)
        self.y = np.array([self.func(x) for x in initial_points]).reshape(-1, 1)
        self.scaler = StandardScaler().fit(self.X)
        self.gpr = GaussianProcessRegressor(kernel=RBF(), alpha=noise**2)

    def surrogate(self, X):
        X_scaled = self.scaler.transform(X)
        return self.gpr.predict(X_scaled, return_std=True)

    def acquisition(self, X):
        X = np.atleast_2d(X)  # Ensure X is 2D
        mu, sigma = self.surrogate(X)
        mu_opt = np.min(self.y)
        z = (mu_opt - mu) / sigma
        ei = (mu_opt - mu) * norm.cdf(z) + sigma * norm.pdf(z)
        return -ei

    def optimize_acquisition(self):
        bounds = [(low, high) for low, high in self.bounds]
        result = minimize(self.acquisition, np.random.uniform(*zip(*self.bounds)), bounds=bounds)
        return result.x

    def suggest_next_point(self, n_iter=1):
        for _ in range(n_iter):
            self.gpr.fit(self.scaler.transform(self.X), self.y)
            next_point = self.optimize_acquisition()
            self.X = np.vstack((self.X, next_point.reshape(1, -1)))
            self.y = np.vstack((self.y, self.func(next_point).reshape(1, -1)))

        return self.X[-1]

# Example usage
def test_function(x):
    x = x[0]
    return (x-2)*(x-5)*(x-8) + np.random.normal(0, 0.1)

ego_optimizer = EGO(func=test_function, bounds=[(0, 10)], initial_points=[[5]])
next_point = ego_optimizer.suggest_next_point()
print("Next point to evaluate:", next_point)


In [ ]:
it = 20
ego_optimizer = EGO(func=test_function, bounds=[(0, 10)], initial_points=[[5]])
fig, ax = plt.subplots(it//2,2, figsize=(10,20))
for i in range(it):
    row_id = i//2
    col_id = i%2
    ax[row_id, col_id].plot(ego_optimizer.X,ego_optimizer.y, "o")
    ego_optimizer.suggest_next_point()

# Data generation

## LHS test

In [ ]:
# import copy
# import math

# def latin_hypercube_sampling(bounds, n_samples):
#     """
#     Generate Latin Hypercube Samples.

#     Parameters:
#         bounds (list of tuples): List of tuples where each tuple represents the lower and upper bounds of the variables.
#         n_samples (int): Number of samples to generate.

#     Returns:
#         np.array: Array of shape (n_samples, n_variables) containing the Latin Hypercube samples.
#     """
#     n_variables = len(bounds)
#     samples = np.zeros((n_samples, n_variables))

#     # Generate quantiles for each variable
#     quantiles = np.linspace(0, 1, n_samples + 1)

#     # Generate random permutation of indices for each variable
#     indices = [list(range(n_samples)) for _ in range(n_variables)]
#     for i in range(n_variables):
#         np.random.shuffle(indices[i])

#     # Assign values from quantiles to the samples
#     for i in range(n_variables):
#         for j in range(n_samples):
#             samples[j, i] = np.random.uniform(quantiles[indices[i][j]], quantiles[indices[i][j] + 1]) * (bounds[i][1] - bounds[i][0]) + bounds[i][0]

#     return samples

# def calculate_objective(samples):
#     """
#     Calculate the objective function.

#     Parameters:
#         samples (np.array): Array of shape (n_samples, n_variables) containing the samples.

#     Returns:
#         float: Value of the objective function.
#     """
#     # Example objective function: sum of squares of differences between samples
#     return np.sum(np.sum(np.diff(samples, axis=0) ** 2))

# def perturb_sample(sample, bounds, temperature):
#     """
#     Perturb a sample using simulated annealing.

#     Parameters:
#         sample (np.array): Array representing the sample to be perturbed.
#         bounds (np.array): Array of shape (n_variables, 2) representing the lower and upper bounds of the variables.
#         temperature (float): Current temperature in simulated annealing.

#     Returns:
#         np.array: Perturbed sample.
#     """
#     perturbed_sample = np.copy(sample)
#     for i in range(sample.shape[1]):  # Iterate over the number of variables (columns)
#         lower_bound, upper_bound = bounds[i]
#         for j in range(sample.shape[0]):  # Iterate over the number of samples (rows)
#             perturbed_sample[j, i] = np.minimum(upper_bound, np.maximum(lower_bound, sample[j, i] + np.random.uniform(-temperature, temperature)))
#     return perturbed_sample

# def simulated_annealing_lhs(bounds, n_samples, max_iterations, initial_temperature, cooling_rate):
#     """
#     Perform Latin Hypercube Sampling (LHS) with Simulated Annealing.

#     Parameters:
#         bounds (list of tuples): List of tuples where each tuple represents the lower and upper bounds of the variables.
#         n_samples (int): Number of samples to generate.
#         max_iterations (int): Maximum number of iterations for simulated annealing.
#         initial_temperature (float): Initial temperature for simulated annealing.
#         cooling_rate (float): Cooling rate for simulated annealing.

#     Returns:
#         np.array: Array of shape (n_samples, n_variables) containing the Latin Hypercube samples.
#     """
#     current_samples = latin_hypercube_sampling(bounds, n_samples)
#     current_objective = calculate_objective(current_samples)
#     best_samples = copy.deepcopy(current_samples)
#     best_objective = current_objective

#     temperature = initial_temperature
#     iteration = 0

#     while iteration < max_iterations and temperature > 0:
#         new_samples = perturb_sample(current_samples, bounds, temperature)
#         new_objective = calculate_objective(new_samples)
#         delta_objective = new_objective - current_objective

#         if delta_objective < 0 or random.random() < math.exp(-delta_objective / temperature):
#             current_samples = new_samples
#             current_objective = new_objective

#         if current_objective < best_objective:
#             best_samples = copy.deepcopy(current_samples)
#             best_objective = current_objective

#         temperature *= cooling_rate
#         iteration += 1

#     return best_samples

# # Example usage:
# bounds = np.array([(0, 10), (-5, 5)])  # Example bounds for three variables
# n_samples = 10  # Number of samples
# max_iterations = 1000  # Maximum number of iterations for simulated annealing
# initial_temperature = 100  # Initial temperature for simulated annealing
# cooling_rate = 0.90  # Cooling rate for simulated annealing

# samples = simulated_annealing_lhs(bounds, n_samples, max_iterations, initial_temperature, cooling_rate)
# # print(samples)
# plt.plot(samples[:,0], samples[:,1], "o")
# plt.show()

In [ ]:
def sequential_adaptive_lhs(current_samples, bounds, variance_grid, max_iterations, initial_temperature, cooling_rate):
    """
    Perform Latin Hypercube Sampling (LHS) with Sequential Adaptive Method.

    Parameters:
        current_samples (np.array): Array of shape (n_samples, n_variables) containing the current samples.
        bounds (np.array): Array of shape (n_variables, 2) representing the lower and upper bounds of the variables.
        variance_grid (np.array): Array of shape (n_samples, n_variables) containing the variance for each variable-sample combination.
        max_iterations (int): Maximum number of iterations for sequential adaptive method.
        initial_temperature (float): Initial temperature for simulated annealing.
        cooling_rate (float): Cooling rate for simulated annealing.

    Returns:
        np.array: Array of shape (n_samples, n_variables) containing the Latin Hypercube samples.
    """
    current_objective = calculate_objective(current_samples)
    best_samples = copy.deepcopy(current_samples)
    best_objective = current_objective

    temperature = initial_temperature
    iteration = 0

    while iteration < max_iterations and temperature > 0:
        new_samples = perturb_sample_adaptive(current_samples, bounds, variance_grid, temperature)
        new_objective = calculate_objective(new_samples)
        delta_objective = new_objective - current_objective

        if delta_objective < 0 or random.random() < math.exp(-delta_objective / temperature):
            current_samples = new_samples
            current_objective = new_objective

        if current_objective < best_objective:
            best_samples = copy.deepcopy(current_samples)
            best_objective = current_objective

        temperature *= cooling_rate
        iteration += 1

    return best_samples

def perturb_sample_adaptive(sample, bounds, variance_grid, temperature):
    """
    Perturb a sample using adaptive perturbation.

    Parameters:
        sample (np.array): Array representing the sample to be perturbed.
        bounds (np.array): Array of shape (n_variables, 2) representing the lower and upper bounds of the variables.
        variance_grid (np.array): Array of shape (n_samples, n_variables) containing the variance for each variable-sample combination.
        temperature (float): Current temperature in simulated annealing.

    Returns:
        np.array: Perturbed sample.
    """
    perturbed_sample = np.copy(sample)
    n_samples, n_variables = sample.shape
    for i in range(n_variables):  # Iterate over the number of variables (columns)
        lower_bound, upper_bound = bounds[i]
        for j in range(n_samples):  # Iterate over the number of samples (rows)
            variance = variance_grid[j, i]  # Variance for the current variable-sample combination
            perturbed_sample[j, i] = np.minimum(upper_bound, np.maximum(lower_bound, sample[j, i] + np.random.normal(0, variance) * temperature))
    return perturbed_sample

In [ ]:
variance = np.zeros((100,100))
variance

In [ ]:
# samples = sequential_adaptive_lhs(samples, bounds, variance, max_iterations, initial_temperature, cooling_rate)
# # print(samples)
# plt.plot(samples[:,0], samples[:,1], "o")
# plt.show()

In [ ]:
# current_samples = latin_hypercube_sampling(bounds, 100)
# current_samples
# plt.plot(current_samples[:,0], current_samples[:,1], "o")
# plt.show()

In [ ]:
# current_objective = calculate_objective(current_samples)
# current_objective

## Sampling

In [ ]:
n_samples = 400
grid_n = int(np.sqrt(n_samples))
n_samples = grid_n * grid_n
# Minimal model testing
g = 1.7
B_lim, D_lim = 2.9, 0.4

In [ ]:
# # LHS search
LHS = lhs.LatinHyperCubeStack()
D_grid, B_grid = LHS.sample_stack([(0, D_lim), (0, B_lim)], n_samples)

In [ ]:
from scipy.stats import qmc
sampler = qmc.LatinHypercube(d=2)
sample = sampler.random(n=n_samples)
sample_scaled = qmc.scale(sample, [0, 0], [B_lim, D_lim])
plt.plot(sample_scaled[:,0], sample_scaled[:,1], "o")
plt.show()

In [ ]:
# D_grid, B_grid = np.meshgrid(sample_scaled[:,0], sample_scaled[:,1])

In [ ]:
RS = random.RandomStack()
D_grid, B_grid = RS.sample_stack([(0, D_lim), (0, B_lim)], grid_n)

In [ ]:
# # Grid search
EG = grid.EqualStack()
D_grid, B_grid = EG.sample_stack([(0, D_lim), (0, B_lim)], grid_n)

In [ ]:
# Run miminal model
dB_dt, dD_dt = mm.step(B_grid, D_grid, g, warm_up=0)

In [ ]:
# plt.plot(B_grid, D_grid, "o")
# plt.show()

In [ ]:
# stream.show(D_grid, B_grid, dD_dt, dB_dt, g)

In [ ]:
# surface.show(D_grid, B_grid, dD_dt, dB_dt, D_lim, B_lim)

In [ ]:
import scipy

In [ ]:
# dB_dt

In [ ]:
# scipy.signal.convolve2d(dB_dt, np.ones((2,2)), mode="same",)

In [ ]:
np.ones((2,2))

In [ ]:
B_grid, D_grid

In [ ]:
# col_i = np.argmax(np.sum(dB_dt, axis=0))
# row_i = np.argmax(np.sum(dB_dt, axis=1))
# print(dB_dt, col_i, row_i)
# B_grid[]

In [ ]:
grad_B, x = np.gradient(dB_dt)
grad_B

In [ ]:
np.abs(grad_B)

In [ ]:
np.argmax(np.abs(grad_B))

In [ ]:
n_samples

In [ ]:
B_grid.shape, D_grid.shape, n_samples, dB_dt.shape, dD_dt.shape

In [ ]:
#  ['B', 'D', 'g', 'dB_dt', 'dD_dt']
X = np.column_stack([B_grid.flatten(), D_grid.flatten(), np.repeat(g, (n_samples))])
y = np.column_stack([dB_dt.flatten(), dD_dt.flatten()])
X.shape, y.shape

# Neural network

## Training

In [ ]:
def load_history(uid):
    with open(f"data/history/{uid}.json", "r") as f:
        history = json.load(f)
    return history

def plot_history(history):
    plt.figure(figsize=(12,6))
    plt.title(f"Training history\n\nLR: {history['hp']['learning_rate']}, BS: {history['hp']['batch_size']}, L1: {history['hp']['l1_reg']}")
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.plot(history['loss'], label="Loss")
    plt.plot(history['val_loss'], label="Val Loss")
    val_min = min(history['val_loss'])
    plt.hlines(y=val_min, xmin=0, xmax=len(history['loss']), colors='green', linestyles='--', lw=2, label=f"Min(Val_loss) = {val_min:.3}")
    plt.legend()
    plt.show()
    
# hist = load_history(NN.uid)
# plot_history(hist)

In [ ]:
2 ** 4

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, shuffle=True)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, shuffle=True)

In [ ]:
hp_1 = {
    'units': (9, 27, 81, 162, 324, 648, 1296),
    'act_fun': 'relu',
    'learning_rate': 1E-3,
    'batch_size': 2 ** 4,
    'l1_reg': 1e-5,
    'n_epochs': 200
}

hp_2 = {
    'units': (128, 512, 1024, 4096),
    'act_fun': 'relu',
    'learning_rate': 1E-3,
    'batch_size': 2 ** 5,
    'l1_reg': 1e-4,
    'n_epochs': 400
}

hp_3 = {
    'units': (128, 512, 1024, 4096),
    'act_fun': 'relu',
    'learning_rate': 1E-3,
    'batch_size': 2 ** 4,
    'l1_reg': 1e-4,
    'n_epochs': 400
}

In [ ]:
# NN = neural_network.NeuralNetwork(uid="20240506_1840")
hp = hp_3
NN = neural_network.NeuralNetwork(hp)
NN.paths['model']

NN.train(X_train, y_train, X_val, y_val)

In [ ]:
hist = load_history(NN.uid)
print(hist['hp']['units'])
last_10 = hist['val_loss'][-10:]
print(np.mean(last_10), np.median(last_10))
plot_history(hist)
NN.uid

In [ ]:
hist = load_history("20240516_1336")
print(hist['hp']['units'])
last_10 = hist['val_loss'][-10:]
print(np.mean(last_10), np.median(last_10))
plot_history(hist)

In [ ]:
hist = load_history("20240516_1252")
print(hist['hp']['units'])
last_10 = hist['val_loss'][-10:]
print(np.mean(last_10), np.median(last_10))
plot_history(hist)

In [ ]:
hist = load_history("20240516_1222")
print(hist['hp']['units'])
last_10 = hist['val_loss'][-10:]
print(np.mean(last_10), np.median(last_10))
plot_history(hist)

In [ ]:
X_train.shape

In [ ]:
plt.plot(X_train[:,0], X_train[:,1], "o")

## Prediction

In [ ]:
pred_grid = 200

In [ ]:
D_true, B_true = EG.sample_stack([(0, D_lim), (0, B_lim)], pred_grid)
X_eval = np.column_stack((B_true.flatten(), D_true.flatten(), np.repeat(g, (pred_grid*pred_grid))))

dB_dt, dD_dt = mm.step(B_true, D_true, g, warm_up=0)
y_eval = np.column_stack((dB_dt.flatten(), dD_dt.flatten()))
X_eval.shape, y_eval.shape

In [ ]:
y_pred = NN.model.predict(X_eval)

In [ ]:
mean_squared_error(y_eval, y_pred)

In [ ]:
pred_dB_dt, pred_dD_dt,  = y_pred[:,0].reshape((pred_grid,pred_grid)), y_pred[:,1].reshape((pred_grid,pred_grid))

In [ ]:
D_true.shape, B_true.shape, pred_dD_dt.shape, pred_dB_dt.shape

In [ ]:
stream.show(D_true, B_true, pred_dD_dt, pred_dB_dt, g)

In [ ]:
surface.show(D_true, B_true, pred_dD_dt, pred_dB_dt, D_lim, B_lim)